In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import copy

import torch
import socialforce

_ = torch.manual_seed(43)

(trajnet)=
# TrajNet++

[TrajNet++ is a pedestrian forecasting challenge](https://www.aicrowd.com/challenges/trajnet-a-trajectory-forecasting-challenge) {cite}`kothari2020human`.
This notebook walks through a first attempt to fit to real-world data
from this challenge {cite}`lerner2007crowds`.

## Synthetic Data

In [ ]:
circle = socialforce.scenarios.Circle()
synthetic_scenarios = circle.generate(1)
synthetic_experience = socialforce.Trainer.scenes_to_experience(synthetic_scenarios)

with socialforce.show.track_canvas() as ax:
    socialforce.show.states(ax, synthetic_scenarios[0])

In [ ]:
!ls ../data-trajnet/train/real_data/
trajnet_scenes = list(socialforce.trajnet.Reader('../data-trajnet/train/real_data/crowds_students001.ndjson').scenes())[:2]


In [ ]:
V = socialforce.potentials.PedPedPotentialMLP()
initial_state_dict = copy.deepcopy(V.state_dict())

simulator = socialforce.Simulator(ped_ped=V) 

In [ ]:
def trajnet_to_socialforce_scenario(pxy):
    pxy = torch.from_numpy(pxy)
    velocities = (pxy[1:] - pxy[:-1]) * 2.5  # convert to m/s with FPS
    states = torch.full((pxy.shape[0], pxy.shape[1], 4), float('nan'))
    states[:, :, :2] = pxy
    states[:-1, :, 2:4] = velocities
    states[-1, :, 2:4] = velocities[-1]

    return torch.stack([simulator.normalize_state(state) for state in states], dim=0)


scenarios = [
    trajnet_to_socialforce_scenario(pxy)
    for _, pxy in trajnet_scenes
]
with socialforce.show.track_canvas() as ax:
    socialforce.show.states(ax, scenarios[0])

In [ ]:
true_experience = socialforce.Trainer.scenes_to_experience(scenarios)
print(true_experience[0][0][0], true_experience[0][1][0])

## MLP

We infer the parameters of an MLP to approximate the 1D scalar 
function $\textrm{SF}(b)$ above from synthetic observations.
The `PedPedPotentialMLP` is a two-layer MLP with softplus activations:
\begin{align}
    \textrm{MLP}(b) &= \textrm{Softplus} \;\; L_{1\times5} \;\; \textrm{Softplus} \;\; L_{5\times1} \;\; b
\end{align}
which is written in terms of linear and non-linear operators where
the Softplus operator applies the softplus function on its input from the right
and $L$ is a linear operator (a matrix) with the subscript indicating the 
$\textrm{output features} \times \textrm{input features}$.
This two-layer MLP with 5 hidden units has 10 parameters.


In [ ]:
# moved up

## Inference

We use a standard optimizer from PyTorch (SGD).
You can specify a standard PyTorch loss function for the `Trainer` as well
but here the default of a `torch.nn.L1Loss()` is used.

In [ ]:
# HIDE OUTPUT
# moved up simulator = socialforce.Simulator(ped_ped=V) 
opt = torch.optim.SGD(V.parameters(), lr=1.0)
socialforce.Trainer(simulator, opt).loop(100, synthetic_experience, log_interval=10)
synthetic_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
opt = torch.optim.SGD(V.parameters(), lr=0.1)
loss = torch.nn.SmoothL1Loss(beta=0.1)
socialforce.Trainer(simulator, opt, loss=loss).loop(10, true_experience)
final_state_dict = copy.deepcopy(V.state_dict())

In [ ]:
# HIDE CODE
with socialforce.show.canvas(ncols=2) as (ax1, ax2):
    # V.load_state_dict(initial_state_dict)
    # socialforce.show.potential_1d(V, ax1, ax2, label=r'initial MLP($b$)', linestyle='dashed', color='C0')

    V.load_state_dict(synthetic_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'synthetic MLP($b$)', linestyle='dotted', color='C0')

    V.load_state_dict(final_state_dict)
    socialforce.show.potential_1d(V, ax1, ax2, label=r'TrajNet++ MLP($b$)', color='C0')